Outline:
1. import datasets

2. data cleaning + export cleaned datasets to `datasets/cleaned`

    2.1 clean `datasets/original/crash`
    
    2.2 clean `datasets/original/delay`
    

In [13]:
import pandas as pd
import numpy as np
from numpy import nan as Nan
import matplotlib.pyplot as plt
import requests
from string import digits
import wget # you need to "pip install wget"  
import glob
import time

import plotly.graph_objects as go # use conda to install plotly --YD
import pandas as pd
import plotly.figure_factory as ff
import xml.etree.ElementTree as ET # to read one dataset in XML format

1. import datasets

In [17]:
# A Pleasent Flight
crash_df = pd.read_csv("datasets/original/crash/Airplane_Crashes_and_Fatalities_Since_1908.csv") #data1
etree = ET.parse("datasets/original/crash/AviationData.xml") #data2
delay_df = pd.read_csv("datasets/original/delay/delay_2009.csv") #data3 

2.1 clean datasets/original/crash

clean data2

In [9]:
xml_root = etree.getroot()

interest_columns = ["EventId","EventDate","Location","Country","Latitude","Longitude","AirportCode","InjurySeverity","AircraftDamage",
                    "AircraftCategory","NumberOfEngines","EngineType","Schedule","TotalUninjured","TotalMinorInjuries",
                    "TotalSeriousInjuries","TotalFatalInjuries","WeatherCondition","BroadPhaseOfFlight","RegistrationNumber","PurposeOfFlight"]

NTSB_crash_df = pd.DataFrame(columns=interest_columns) # NTSB_crash_df: dataframe which collects airport information. -- YD 02/28/2020

In [11]:
for elem in xml_root: # This loop will run only once
    for row in elem: 
        if not(row.attrib["PurposeOfFlight"]=="Business"):  # Only care about business flights whose engine type is not 'Reciprocating'-- YD 02/28/2020
            continue
        if row.attrib["EngineType"]=="Reciprocating": 
            continue
        information = list()
        for interest in interest_columns:
            if not (row.attrib[interest]==""):
                information.append(row.attrib[interest])
            else:
                information.append(np.nan)
        row_information = pd.Series(information,index=interest_columns)
        NTSB_crash_df = NTSB_crash_df.append(row_information,ignore_index=True)
            
# this may need to run for a while. It takes 18 seconds on my computer
#     NTSB_crash_df

In [8]:
# NTSB_crash_df[(NTSB_crash_df["AircraftCategory"]=="Airplane") & (NTSB_crash_df["EngineType"]!="Reciprocating") & (NTSB_crash_df["Country"]=="United States")]

In [12]:
# (unfinished Crash Data Cleaning.ipynb)

In [15]:
delay_2009_df = pd.read_csv("datasets/delay/2009.csv")

# DEPARTURE
unpleasant_2009_departure = pd.DataFrame()
unpleasant_2009_departure['total_departure'] = delay_2009_df.loc[:,["ORIGIN"]].groupby('ORIGIN').size()
unpleasant_2009_departure[["average_departure_delay","average_departure_taxi"]] = delay_2009_df.loc[:,["ORIGIN","DEP_DELAY","TAXI_OUT"]].groupby('ORIGIN').mean()
unpleasant_2009_departure['average_departure_cancelled'] = delay_2009_df.loc[:,["ORIGIN","CANCELLED"]].groupby('ORIGIN').mean()
unpleasant_2009_departure['averge_departure_distance'] = delay_2009_df.loc[:,["ORIGIN","DISTANCE"]].groupby('ORIGIN').mean()
    
#ARRIVAL
unpleasant_2009_arrival = pd.DataFrame()
unpleasant_2009_arrival['total_arrival'] = delay_2009_df.loc[:,["DEST"]].groupby('DEST').size()
unpleasant_2009_arrival[["average_arrival_delay","average_arrival_taxi"]] = delay_2009_df.loc[:,["DEST","ARR_DELAY","TAXI_IN"]].groupby('DEST').mean()
unpleasant_2009_arrival['average_arrival_diverted'] = delay_2009_df.loc[:,["DEST","DIVERTED"]].groupby('DEST').mean()
unpleasant_2009_arrival['averge_arrival_distance'] = delay_2009_df.loc[:,["DEST","DISTANCE"]].groupby('DEST').mean()
    
unpleasant_2009_departure['total_departure_lg10'] = unpleasant_2009_departure['total_departure'].apply(np.log10)
unpleasant_2009_arrival['total_arrival_lg10'] = unpleasant_2009_arrival['total_arrival'].apply(np.log10)

unpleasant_2009 = unpleasant_2009_departure.merge(unpleasant_2009_arrival,left_index=True,right_index=True)
unpleasant_2009.to_csv("processed/unpleasant_2009.csv")


FileNotFoundError: [Errno 2] File b'datasets/delay/2009.csv' does not exist: b'datasets/delay/2009.csv'